In [92]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import numpy as np

In [ ]:
df = pd.read_csv('/content/full_FEVS_prototype.csv')
df.head()

# Functions


In [94]:
reporting_levels = ['Reporting Level_2015', 'Reporting Level_2016', 'Reporting Level_2017', 'Reporting Level_2018', 'Reporting Level_2019', 'Reporting Level_2020', 'Reporting Level_2022']

response_counts = ['Response Count_2015', 'Response Count_2016', 'Response Count_2017', 'Response Count_2018', 'Response Count_2019', 'Response Count_2020', 'Response Count_2022']

employee_engagement_overall = ['Employee Engagement: Overall_2015', 'Employee Engagement: Overall_2016', 'Employee Engagement: Overall_2017', 'Employee Engagement: Overall_2018', 'Employee Engagement: Overall_2019', 'Employee Engagement: Overall_2020', 'Employee Engagement: Overall_2022']

employee_engagement_leaders_lead = ['Employee Engagement: Leaders Lead_2015', 'Employee Engagement: Leaders Lead_2016', 'Employee Engagement: Leaders Lead_2017', 'Employee Engagement: Leaders Lead_2018', 'Employee Engagement: Leaders Lead_2019', 'Employee Engagement: Leaders Lead_2020', 'Employee Engagement: Leaders Lead_2022']

employee_engagement_supervisors = ['Employee Engagement: Supervisors_2015', 'Employee Engagement: Supervisors_2016', 'Employee Engagement: Supervisors_2017', 'Employee Engagement: Supervisors_2018', 'Employee Engagement: Supervisors_2019', 'Employee Engagement: Supervisors_2020', 'Employee Engagement: Supervisors_2022']

employee_engagement_intrinsic = ['Employee Engagement: Intrinsic Work Experience_2015', 'Employee Engagement: Intrinsic Work Experience_2016', 'Employee Engagement: Intrinsic Work Experience_2017', 'Employee Engagement: Intrinsic Work Experience_2018', 'Employee Engagement: Intrinsic Work Experience_2019', 'Employee Engagement: Intrinsic Work Experience_2020', 'Employee Engagement: Intrinsic Work Experience_2022']

new_iq_overall = ['New IQ: Overall_2015', 'New IQ: Overall_2016', 'New IQ: Overall_2017', 'New IQ: Overall_2018', 'New IQ: Overall_2019']

new_iq_fair = ['New IQ: Fair_2015', 'New IQ: Fair_2016', 'New IQ: Fair_2017', 'New IQ: Fair_2018', 'New IQ: Fair_2019']

new_iq_open = ['New IQ: Open_2015', 'New IQ: Open_2016', 'New IQ: Open_2017', 'New IQ: Open_2018', 'New IQ: Open_2019']

new_iq_cooperative = ['New IQ: Cooperative_2015', 'New IQ: Cooperative_2016', 'New IQ: Cooperative_2017', 'New IQ: Cooperative_2018', 'New IQ: Cooperative_2019']

new_iq_supportive = ['New IQ: Supportive_2015', 'New IQ: Supportive_2016', 'New IQ: Supportive_2017', 'New IQ: Supportive_2018', 'New IQ: Supportive_2019']

new_iq_empowered = ['New IQ: Empowered_2015', 'New IQ: Empowered_2016', 'New IQ: Empowered_2017', 'New IQ: Empowering_2018', 'New IQ: Empowering_2019']

global_satisfaction = ['Global Satisfaction_2015', 'Global Satisfaction_2016', 'Global Satisfaction_2017', 'Global Satisfaction_2018', 'Global Satisfaction_2019', 'Global Satisfaction_2020', 'Global Satisfaction_2022']

# 2D array for use in analysis functions
variables_list = [employee_engagement_leaders_lead, employee_engagement_supervisors, employee_engagement_intrinsic, new_iq_overall, new_iq_fair, new_iq_open, new_iq_cooperative, new_iq_supportive, new_iq_empowered, global_satisfaction ]




In [125]:

#Helper function to turn the variable name into string for naming in column
def var_name(variable):
    # Get the globals() dictionary
    global_vars = globals()

    # Iterate over the items in globals()
    for name, value in global_vars.items():
        if value is variable:
            return name


#Takes a list of columns, creates a series of means and barplots them
def barplot_select_column(df, selected_columns):
    column_means = df[selected_columns].mean(axis=0)

    # Create the bar plot
    column_means.plot(kind='bar')

    # Add labels and title
    plt.ylabel(f'Mean {var_name(selected_columns)}')
    plt.title(f'Mean {var_name(selected_columns)} Scores by Year')

    # Display the plot
    plt.show()



#Takes a series and return True if meets standards of normality, used a filter in the Anova Test Function
def test_normality(df):

        # Perform Shapiro-Wilk test for normality
        statistic, p_value = stats.shapiro(df)

        # Check the significance level to interpret the results
        alpha = 0.05
        if p_value > alpha:
           return True


#Takes a list of columns, creates a new dataframe from them, then preforms ANOVAs on each year compared to the next and prints the results
def anova_test(df, variables):
  variable_df = df[variables]
  #test the year of the variable
  print(var_name(variables).upper() + '\n')
  for year in variables:

    if (variables.index(year) != len(variables) - 1) and test_normality(variable_df[year]) and  test_normality(variable_df[variables[variables.index(year)+1]]):
        f_statistic, p_value = stats.f_oneway(variable_df[year], variable_df[variables[variables.index(year)+1]])

        #Print the results
        print(year + ' against ' + variable_df[variables[variables.index(year)+1]].name)
        print("F-statistic:", f_statistic)
        print("p-value:", p_value)
        print('\n')
        if p_value < 0.05:
            print("The p-value is less than 0.05, indicating a significant difference between the groups.")
        else:
            print("The p-value is greater than or equal to 0.05, indicating no significant difference between the groups.")
        print('\n')


        print('---------------------------------------------------------------')



#Takes dataframe as input and creates heatmaps of variables for each year
def heatmap_years(df):
    years = ['2015', '2016', '2017', '2018', '2019', '2020', '2022']
    year_dfs = []

    for year in years:
        year_df = pd.DataFrame()  # Create a DataFrame for the column
        for column in df.columns:
            if year in column and (df[column].dtype != 'object'):
                year_df.insert(loc=len(year_df.columns), column=column, value=df[column])
        year_dfs.append(year_df)  # Store the year and DataFrame tuple

    # Create and display the heatmaps
    for heatmap_df, year in zip(year_dfs, years):
        # Compute the correlation matrix
        corr_matrix = heatmap_df.corr()

        mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
        # Create a heatmap
        plt.figure(figsize=(10, 8))
        sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', linewidths=0.5, mask=mask)
        plt.title(f'Variable Comparison Heatmap {year}')
        plt.show()

        #sort by correlation strength and show the list of strongest correlations
        sorted_corr_matrix = corr_matrix.reindex(corr_matrix.abs().sum().sort_values(ascending=False).index, axis=0)
        top_correlated_variables = sorted_corr_matrix.columns[:]
        # Print the results
        i = 1
        print(f"The sorted correlated variables are:")
        for variable in reversed(top_correlated_variables):
          print(f'{i}. ' + variable)
          i = i + 1


#Takes a dataframe and for each agency, plots each key varibles for each year
def agency_key_inspection(df):
  #list for iteration
  score_types = [employee_engagement_intrinsic, new_iq_empowered, global_satisfaction]

  for agency in df['Agency & Subagency Name']:

    # select row based on agency name
    agency_row = df.loc[df['Agency & Subagency Name'] == agency]

    for score_type in score_types:
      # I want a dataframe that is the yearly scores for the agency for each score

      #create a frame that is
      scores = agency_row[score_type].squeeze()

      # Create the bar plot
      scores.plot(kind='bar')


      # Add labels and title
      plt.ylabel(f'{var_name(score_type)} Scores by Year')
      plt.title(f'{agency}')

      # Customize the appearance of the plot
      plt.grid(axis='y', linestyle='--')  # Add gridlines for the y-axis
      plt.show()

    print('---------------------')
  print('-------------------------------------------------------------')


def key_score_plot(df):
  #list for iteration
  score_types = [employee_engagement_intrinsic, new_iq_empowered, global_satisfaction]
  #list for tracking of change
  change_list = []
  for agency in df['Agency & Subagency Name']:

    # select row based on agency name
    agency_row = df.loc[df['Agency & Subagency Name'] == agency]

    # I want a series that is the average of these three key scores for each year
    key_df = {}

    #loop for each year in the lists
    for i in range(len(global_satisfaction)):

      key = 0
      #keeps track for if we have to pass a variale
      count = 0


      #loop for each variable type, access the next years score

      for score in score_types:
        try:
          key = key + df.loc[df['Agency & Subagency Name'] == agency][score[i]]
          count = count + 1
        except:
          continue


      #create a column for the year
      key_df[f'{score[i][-4:]}'] = key / count #year and value of key value



    # Create the bar plot for the agency

    key_df = pd.DataFrame.from_dict(key_df)
    key_df = key_df.squeeze()

    key_df.plot(kind='line')


    # Add labels and title
    plt.ylabel('Key Scores by Year')
    plt.title(f'{agency}')

    # Customize the appearance of the plot
    plt.grid(axis='y', linestyle='--')  # Add gridlines for the y-axis
    plt.show()

    # Calculate the difference between the last and first values in the series
    difference = key_df.iloc[-1] - key_df.iloc[-2]

    # Print the difference
    print("Difference from 2020 to 2022: " + str(difference))
    change_list.append((agency, difference))
    print('---------------------')

  # Sort the list of tuples by the second element (integer)
  sorted_change_list = sorted(change_list, key=lambda x: x[1])
  sorted_change_list = sorted(sorted_change_list , key=lambda x: x[1], reverse=True)
  for value in sorted_change_list:
    print(value)
    print('\n')








  print('-------------------------------------------------------------')



# Visualizing Scores by Year

- There seems to be a shared trend of increasing scores that tapers off and possibly lowers after 2020
- Scores may be positively correlated to eachother since they are increasing together


In [ ]:
# iterates over each list of variables and preforms barplotting on the years for it
for variable in variables_list:
  barplot_select_column(df,variable)

# Score Trends



In [ ]:
'''
AVOVA Tests:

Hypothesis
- Null hypothesis (H₀): The average scores are the same across all years.
- Alternative hypothesis (H₁): The average scores are different across at least two years.

Data Organization
- Each year is represented by a separate column in the dataset, with the variables for each year recorded in the respective columns.

Assumption Checking
- ANOVA assumes independence of observations, which is met by nature of survey data and normality of the data within each group, which is checked by the test_normality() function

'''

for variable in variables_list:
  barplot_select_column(df,variable)
  anova_test(df, variable)


## Trend Insights
- There is a trend in the data that shows a subtle increase in survey scores from 2015 to 2018, which then tapers off in some variables until 2020, where there is a general increase in scores. After this there is a decrease in 2022.

- This is supported by ANOVAs preformed on each grouping of variables which indicates signifigant differences between years of increase/decrease and inidicates lack of signifigant differences in plateau years.

- The cause of this trend could be attributed to many factors, which could be explored via analysis of policy across government, specifically NOAA agencies.

- Policy or economic factors most likely influenced the trend of a rise in metrics.

- A possible factor in the decrease of scores could be attributed to COVID and compared against policies that were affected to guide ways to increase scores.


# Correlations of Scores

Heatmaps visualize the correlation between variables. Darker colors represent stronger positive or negative correlations, while lighter colors indicate weaker or no correlations.

There are strong correlations between related variables and very weak correlations between response counts to the rest of the data, which indicates this is coherent data that makes sense on an intuitive level.

- For each year a heatmap has been created to show the strength of the correlations between variables
- Each heatmap also features a list the variables sorted in order of overall correlation to the other variables of the dataset, which is an indicator of their importance in the dataset

In [ ]:
heatmap_years(df)

## Correlation Insights
- Employee Engagement: Intrinsic Work Experience and New IQ: Empowered were  high in importance compared to other similar metrics, shown by their strong positive correlation to the other variables. This is most likely due to their intrinsic meanings.

### Employee Engagement: Intrinsic Work Experience

Employee Engagement: Intrinsic Work Experience reflects the level of intrinsic motivation and satisfaction employees derive from their work.

- **Significance**: Intrinsic Work Experience plays a crucial role in shaping employee engagement levels and job satisfaction.
- **Impact**: When employees find their work meaningful, fulfilling, and aligned with their values, it positively influences their overall engagement, productivity, and job performance.
- **Strategies**: Organizations can enhance Intrinsic Work Experience by providing challenging and meaningful work assignments, promoting autonomy and decision-making authority, recognizing and leveraging employees' strengths, and offering opportunities for personal and professional growth.

### New IQ: Empowered

New IQ: Empowered measures the level of empowerment individuals perceive within the organization.

- **Importance**: Empowerment is a key aspect of fostering employee engagement and organizational success.
- **Impact**: When individuals feel empowered, they are more likely to take initiative, make decisions, and contribute effectively to the organization's goals.
- **Strategies**: Organizations can enhance empowerment by providing employees with autonomy, decision-making authority, clear communication channels, and opportunities for growth and development.

### What can we take away from this?
The importance of "Employee Engagement: Intrinsic Work Experience" and "New IQ: Empowered" variables in survey data signifies the importance of fostering a culture of personal fulfillment and empowerment among its employees.

Employees who perceive a high level of intrinsic work experience, finding their work meaningful and aligned with their values, are more likely to be engaged and satisfied in their roles. This makes sense as NOAA's mission is environmental conservation and scientific research. Tapping into employees' passion for protecting our oceans and atmosphere will increase sentiment towards work and increase productivity.

Additionally, empowering employees by providing resources, support, and autonomy allows them to make informed decisions and drive innovation within the organization. This empowerment fosters a culture of collaboration, problem-solving, and continuous improvement, enabling NOAA to address complex environmental challenges effectively.


# Agency Trends in Key Scores


Agencies have visable difference in key score trends, which tells us that different agencies experienced varied scores across our timeframe and that not all agencies experienced a drop in scores.

In [ ]:
agency_key_inspection(df)

## Analyzing Mean Key Score Change by Office
Scores dropped signifigantly from 2020 to 2022 across agencies. It was found that the variables correlated to other variables the most were Employee Engagement: Intrinsic Work Experience, New IQ: Empowered and Global Satisfaction. We can use these variables to find agencies who improved their scores during the downward trend.
- The mean key score is a feature that combines an agencies (Global Satisfaction), (Employee Engagement: Intrinsic Work Experience) and (New IQ: Empowered) scores by taking their average.
- Line plotting is used help to identify the agencies whose mean key score has increased in recent years.
- Agencies are then sorted in order of increase in Key Score from 2020 and 2022.

In [ ]:
key_score_plot(df) #Create a list of changes in scores by agency for this

### Mean Key Score Change Insights
Based on the provided list of offices and their corresponding score changes from 2020 to 2022, here are the top 5 offices with the highest score changes:

1. Office of Projects, Partnerships, and Analysis: 0.111287095
2. Aviation Operations Center: 0.08117562999999994
3. Office of Protected Resources: 0.021850029999999965
4. National Center for Environmental Information: 0.011025404999999933
5. Office of Sustainable Fisheries: 0.01047330000000013

These offices experienced the most positive changes in satisfaction scores in 2022 compared to the previous year in 2020.
